In [1]:
import cvrplib
import matplotlib.pyplot as plt
import numpy as np
import random
import re
import sys
iters_limit = 50000
iters_stagnation_limit = 3000

In [2]:
# Download instances
instance, solution = cvrplib.download('B-n31-k5', solution=True)


In [3]:
def data_preparation(number_points):
    
    most_consuming_edge = 0
    for i in range(number_points):
        for j in range(number_points):
            if distance_matrix[i][j] > most_consuming_edge:
                most_consuming_edge = distance_matrix[i][j]
        

    tay = ((number_points)*most_consuming_edge)**(-1)
    return(tay)

In [4]:
def super_way(route,minimal_route_cost):
    colors =["green","blue","grey","orange","red","pink","cyan","olive","brown","black","Purple","darkkhaki","darkmagenta","honeydew","lightcoral","lime"]
    fig, (ax1,ax2) = plt.subplots(1,2,figsize=(15,7))
    fig.suptitle(instance.name)
    for j in range(len(solution.routes)):
        x=[instance.coordinates[0][0]]
        y=[instance.coordinates[0][1]]
        for i in solution.routes[j]:
            x+=[instance.coordinates[i][0]]
            y+=[instance.coordinates[i][1]]
        x+=[instance.coordinates[0][0]]
        y+=[instance.coordinates[0][1]]
        ax1.plot(x,y)
        ax1.set_title(f'Правильное решение\nМашин:{instance.name[-1]} \nСтоимость {solution.cost}')

    ax1.scatter(*instance.coordinates[instance.depot],color = "black")
    for i in instance.customers:
        ax1.scatter(*instance.coordinates[i],color = "black")
    def way(route,minimal_route_cost,ax):
        o = -1
        x = [instance.coordinates[0][0]]
        y = [instance.coordinates[0][1]]
        for i in route:
            x += [instance.coordinates[i][0]]
            y += [instance.coordinates[i][1]]
            #print(x,y)
            ax.plot(x,y,color=colors[o+1])
            ax.scatter(x,y, color="blue")
            if i == instance.depot:
                x = [instance.coordinates[i][0]]
                y = [instance.coordinates[i][1]]
                o+=1
        ax.scatter(*instance.coordinates[instance.depot],color = "black")
        ax.set_title(f'Мое решение\nМашин: {o} \nСтоимость {minimal_route_cost}')
        return(ax,o)
    ax2,o = way(route,minimal_route_cost,ax2)
    f = open('123.txt', 'a')
    f.write(instance.name+" ")
    f.write(instance.name[-1]+" ")
    f.write(str(solution.cost)+" ")
    f.write(str(o)+" ")
    f.write(str(minimal_route_cost)+" ")
    f.write('\n')
    f.close()
    fig.show()
    fig.savefig(instance.name)

In [5]:
def pheromone_update(matrix,a,p,L):
    for i in range(number_points-1):
        for j in range((i+1),number_points):
                matrix[i][j] = matrix[j][i] = matrix[i][j]*(1 - p)+p*tao
    #for k in a:
        #matrix[a[k]][a[k+1]] = matrix[a[k+1]][a[k]] = matrix[a[k]][a[k+1]]/(1 - p) + 1/L

In [6]:
def ant_alg(alpha, beta, qO,p,k1):
    best_route = []
    minimal_route_cost = 50000
    iter = 1
    stagnation_iter = 0
    
   
    while (iter <= iters_limit) and (stagnation_iter <= iters_stagnation_limit):
        #if (iter-1)%100 == 0:
            #print(f'Итерация {iter} из {iters_limit}. Лучшая стоимость:{minimal_route_cost}')
        for ant in range(50):
            current_route = [0]
            non_visited_points = list(range(1,number_points+1))
            #print(non_visited_points)
            #print(k1)
            for z in range(1,len(k1)):
                non_visited_points.remove(k1[z])
            capacity_remains = capacity
            avto=1
            #print((non_visited_points))
            while len(non_visited_points)>0:
                current_point = current_route[-1]
                feromon_values = np.array([ (pheromon_matrix[current_point][point]**(alpha))*((1/distance_matrix[current_point][point])**beta) for point in non_visited_points if (point not in current_route and distance_matrix[current_point][point]!=0)])
                
                if np.random.random_sample() < qO:
                    next_point = non_visited_points[np.argmax(feromon_values)]
                    
                else:
                    next_point = np.random.choice(non_visited_points, p=feromon_values/sum(feromon_values))

                capacity_remains -= demand[next_point]
                if capacity_remains < 0:
                    current_route.append(0)
                    capacity_remains = capacity
                    avto+=1
                else:
                    non_visited_points.remove(next_point)
                    current_route.append(next_point)
                #print(non_visited_points)
                    
            current_route.append(0)
            iter += 1
            current_route_cost = 0
            
            for i in range(1, len(current_route)):
                p1, p2 = current_route[i-1], current_route[i]
                pheromon_matrix[p1][p2] = pheromon_matrix[p2][p1] = pheromon_matrix[p1][p2]*(1 - alpha) + alpha*tay
                current_route_cost += distance_matrix[p1][p2]
            
            if current_route_cost < minimal_route_cost:
                best_route = current_route.copy()
                minimal_route_cost = current_route_cost
                stagnation_iter += -1
            else:
                stagnation_iter += 1
            
              
        
        for i in range(1, len(best_route)):
            p_1, p_2 = best_route[i-1], best_route[i]
            pheromon_matrix[p_1][p_2] = pheromon_matrix[p_2][p_1] = pheromon_matrix[p_1][p_2]*(1-alpha) + 10*alpha/current_route_cost
        
        
    print(iter)
    print(stagnation_iter)
    return(best_route, minimal_route_cost,avto)

In [ ]:
capacity,points,number_points,demand = instance.capacity,instance.customers,instance.n_customers,instance.demands
k=[0]
for i in range(1,len(points)+1):
    for j in range(i+1,len(points)+1):
        if (instance.distances[i][j]==0):
            k.append(j)
            print("i=",i,"j= ",j,"demand=",demand[i],"  ",demand[j])
            demand[i]=instance.demands[i]+instance.demands[j]
            print("posle demand=",demand[i])
print(k)

distance_matrix=tuple(instance.distances)
tay = data_preparation(number_points)
pheromon_matrix = tuple([tay for _ in range(number_points+1)] for _ in range(number_points+1))
best_route, minimal_route_cost,avto = ant_alg(alpha=2, beta=3, qO=.4,p=.6,k1=k)

i= 7 j=  30 demand= 22    19
posle demand= 41
i= 13 j=  17 demand= 16    15
posle demand= 31
[0, 30, 17]


In [1]:
super_way(best_route,minimal_route_cost )


NameError: name 'super_way' is not defined